# Week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

NOTE: Using Openrouter for the openai API and llama3.2:1b for Ollama

In [10]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [11]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'


OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [12]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key!")
    
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8888",  # or your app/site URL
        "X-Title": "llm_engineering_course"       # optional label
    }
)

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

API key looks good so far


In [13]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [14]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can answer questions about code."},
            {"role": "user", "content": question}
          ],
        stream=True
    )   

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

This line of code is using a combination of Python features: a generator expression (`yield from`) and a set comprehension. Let's break down what it does step by step:

1. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It creates a set of authors by iterating over a collection called `books`.
   - For each `book` in `books`, it attempts to retrieve the value associated with the key `"author"` using `book.get("author")`.
   - The `if book.get("author")` condition filters out any `book` entries where the author is `None` or an empty string (i.e., it only includes books that have a valid author).

2. **Yield from**:
   - The `yield from` statement is used in a generator function to yield all values from another iterable. In this case, it yields each author from the set created by the comprehension.
   - Note that using `yield from` with a set means that the function will yield each unique author, one at a time.

### Summary
In summary, the code effectively generates an iterable of unique author names from a collection of `books`. It ensures that only those authors that are non-empty and valid are considered. This is useful in scenarios where authors may be duplicated across multiple books, and you want to avoid yielding the same author multiple times. 

### Why It’s Useful
- **Efficiency**: It collects unique authors without having to manually check for duplicates, thanks to the properties of a set.
- **Readability**: The use of comprehensions and `yield from` creates a concise and readable expression of the intention to collect and yield unique authors.

In [15]:
# Get Llama 3.2 to answer
ollama_stream = ollama.chat.completions.create(model=MODEL_LLAMA, messages=[{"role": "user", "content": question}], stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in ollama_stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

This is a Python code snippet that uses the `yield from` expression to generate an iterator of authors' names. Here's a breakdown of how it works:

1. `{book.get("author") for book in books}`: This part of the code creates a generator expression that iterates over each book in the `books` collection and gets its author's name (key `author`) using the `.get()` method.
2. `yield from`: This is the key part of the code. When this symbol is encountered, the code will start yielding control to the next line of the generator expression.

Key aspects:

* The yield keyword in Python is used to produce values over a long period of time, typically in connection with generators, which are special types of functions that can be paused and resumed.
* `yield from` allows an iterable (like a list or tuple) to be short-circuited, i.e., it produces values until the end is reached.

The purpose of this code snippet likely appears when you're working with large datasets like JSON data loaded into Python. Let's say `books` is a collection of dictionaries, each representing a book in your dataset, and their respective authors (`author`). You're interested in getting only these author names where the value matches an expected key.

When used inside a context manager or within another generator function, this code snippet can be quite useful:

- It uses the `yield from` expression to avoid the extra `get()` call for each book.
- It doesn't require manual iteration through all books in Python's memory; instead, it generates values on-the-fly and can skip some books due to what you want.